# Exercise: Can your favourite handball team still win?
<font color='blue'><b>Goal:</b></font>
Implement the algorithm in Chapter 4.4.8 in the script and obtaining the certificates seen in class; testing the implementation on a real-world example.

<font color='blue'><b>Python packages required:</b></font> `networkx`

## A real world-example: HBL season 2017/18

HBL is the top German professional handball league, where 18 teams play each other, earning two points for each game they win and one point for each draw. On February 18, 2018, after the 24th of a total of 34 matchdays was complete, the full table was the following.

| Rank | Team                   | Points |
|------|------------------------|--------|
|  1   | Rhein-Neckar Löwen     | 40     |
|  2   | Füchse Berlin          | 40     |
|  3   | SG Flensburg-Handewitt | 38     |
|  4   | TSV Hannover-Burgdorf  | 37     |
|  5   | SC Magdeburg           | 35     |
|  7   | THW Kiel               | 31     |
|  6   | MT Melsungen           | 31     |
|  8   | SC DHfK Leipzig        | 29     |
|  9   | HSG Wetzlar            | 23     |
| 10   | TBV Lemgo              | 21     |
| 11   | FRISCH AUF! Göppingen  | 20     |
| 12   | TSV GWD Minden         | 19     |
| 13   | HC Erlangen            | 16     |
| 14   | TVB 1898 Stuttgart     | 13     |
| 15   | VfL Gummersbach        | 12     |
| 16   | TuS N-Lübbecke         | 11     |
| 17   | TV 05/07 Hüttenberg    |  9     |
| 18   | Die Eulen Ludwigshafen |  7     |

Your German friend Max is a big fan of the team "TBV Lemgo" (currently ranked 10th), the German champion of 1997 and 2003. Of course he's hoping for another German championship - and if you ask him, there is still hope in this season: His team currently has 21 points with 10 matchdays remaining (on every matchday, every team plays precisely one match), so "TBV Lemgo" could end up with 41 points, which is more than the current leaders "Rhein-Neckar Löwen" and "Füchse Berlin" have now. To be on the safe side, he even checked that these two teams do not play each other in the remaining season.

<font color="blue"><b>Your first task:</b></font> Do you agree with Max's arguments? Why, or why not? Is there still hope for "TBV Lemgo" to become the German champion 2017/18? Can this be decided from the information given above?

## The general setting

In general, you are given the following information:
- A list of the teams and their current points. You may assume that this information is given to you as a dictionary with teams as keys, and their current points as values.
- The remaining games of the season. As seen in the script, these can be encoded in a (multi-)graph, where each team corresponds to a vertex, and an edge connecting two teams indicates that they will play each other in the remaining season (note: as there might potentially be two or more games among two teams, we need a multigraph here). We always assume that for a draw, both teams involved get one point; else the winning team gets two and the losing team no points.
- A team for which you want to know whether there is a chance that the team finishes the season as a sole leader, i.e., with strictly more points than every other team.

<font color="blue"><b>Your second task:</b></font> Implement the function `can_they_win(current_points, remaining_games_graph, team)` that returns whether `team` can be a sole leader at the end of the season if the current points are given by the dictionary `current_points`, and the remaining games of the season are encoded in the (multi-)graph `remaining_games_graph`.

*Hint: Section 4.4.8 in the script (as discussed in the lecture) provides some guidance!*

In [30]:
import networkx as nx
def can_they_win(current_points, remaining_games_graph, team):

    ## assume team winns everything
    possible_points = current_points[team] + len(remaining_games_graph.edges(team))
    st_graph = nx.DiGraph()
    st_graph.add_node("s")
    st_graph.add_node("t")
    for h in current_points:
        if (team == h):
            continue
        st_graph.add_node(h)
        st_graph.add_edge(h,"t", capacity=possible_points - current_points[h])

    i = 0
    for edge in remaining_games_graph.edges:
        if (team == edge[0] or team == edge[1]):
            continue
        game_name = f'g_{i}'
        st_graph.add_node(game_name)
        st_graph.add_edge("s", game_name, capacity=2)
        st_graph.add_edge(game_name, edge[0], capacity=2)
        st_graph.add_edge(game_name, edge[1], capacity=2)
        i += 1


    min_cut = nx.minimum_cut(st_graph,"s","t")
    return  min_cut[0] == 2*i

## Testing your implementation

The module `winningPossibilities_module.py` contains helper functions that return the information about the HBL season 2017/18 for any matchday:
- `get_current_points(m)` returns a dictionary with the teams as keys and their points as values after matchday `m`.
- `get_remaining_games_graph(m)` returns a (multi-)graph with the teams as vertices and edges representing games of the remaining season after matchday `m`.
  
  *Note: To distinguish parallel edges in multigraphs,* `networkx` *uses internal keys. Thus, if you go through the edges of a multigraph* `g`*, you might want to use* `g.edges(keys = True)`*, which gives edges as triples in the form* `(endpoint1, endpoint2, key)`*, and thus lets you distinguish parallel edges by the key.*
- `print_table(m)` prints the table after matchday `m`.*

For example, the following command replicates the table after matchday 24 given in the introduction above.

In [3]:
import winningPossibilities_module as helpers

helpers.print_table(24)

Table after matchday 24:
+------+------------------------+--------+
| Rank | Team                   | Points |
+------+------------------------+--------+
|    1 | Füchse Berlin          |     40 |
|    2 | Rhein-Neckar Löwen     |     40 |
|    3 | SG Flensburg-Handewitt |     38 |
|    4 | TSV Hannover-Burgdorf  |     37 |
|    5 | SC Magdeburg           |     35 |
|    6 | THW Kiel               |     31 |
|    7 | MT Melsungen           |     31 |
|    8 | SC DHfK Leipzig        |     29 |
|    9 | HSG Wetzlar            |     23 |
|   10 | TBV Lemgo              |     21 |
|   11 | FRISCH AUF! Göppingen  |     20 |
|   12 | TSV GWD Minden         |     19 |
|   13 | HC Erlangen            |     16 |
|   14 | TVB 1898 Stuttgart     |     13 |
|   15 | VfL Gummersbach        |     12 |
|   16 | TuS N-Lübbecke         |     11 |
|   17 | TV 05/07 Hüttenberg    |      9 |
|   18 | Die Eulen Ludwigshafen |      7 |
+------+------------------------+--------+


<font color="blue"><b>Your third task:</b></font> Apply your function to test whether Max can indeed still hope for a German championship after matchday 24 of the 2017/18 season. What about fans of "HSG Wetzlar" (currently ranked 9th), is there a way that their team will be the sole leader at the end of the season? If you want, you can also test other matchdays and teams using the provided helper functions.

In [35]:
# Can "TBV Lemgo" be the sole leader at the end of the season?
current_points = helpers.get_current_points(24)
matches = helpers.get_remaining_games_graph(24)
print(can_they_win(current_points, matches, "TBV Lemgo"))

# Can "HSG Wetzlar" be the sole leader at the end of the season?
current_points = helpers.get_current_points(24)
matches = helpers.get_remaining_games_graph(24)
print(can_they_win(current_points, matches, "HSG Wetzlar"))


False
False


## Getting certificates

Now that you programmed your function, you can tell your friend Max that his hopes are illusory - his team "TBV Lemgo" will not be the sole leader at the end of the season. Just telling him this, though, he does not believe you: He did the calculations and his arguments seem convincing to him.

<font color="blue"><b>Your fourth task:</b></font> Can you modify your function to also provide a certificate if the given team cannot end the season as the sole leader? How would you explain this certificate to your friend Max?

*Hint: A way to obtain such certificates was discussed in the lecture!*

In [37]:
def can_they_win_certificate(current_points, remaining_games_graph, team):

    ## Your code goes here.

    return (True/False, certificate)